In [1]:
#for j in $(seq 3 4); do for i in $(seq 1 20); do python main --seed $((20 * j + $i))&done;done;

In [2]:
import bempp.api
import numpy as np

import sys
sys.path.append('../../')
from decimal import *

import bemppUQ
from bemppUQ.operators.maxwell import assemble_operators, evaluate_far_field, evaluate_far_field_sd

bempp.api.global_parameters.assembly.potential_operator_assembly_type = 'dense'
bempp.api.global_parameters.hmat.eps = 1E-4


from bemppUQ.shapes import reentrant_cube_exterior

def function(x, y, i, j):
    zx = 0
    zy = 0
    if i == 0:
        zx = np.sin(x * np.pi * 2)
    if i == 1:
        if x <= 0.25:
            zx = np.sin(x * np.pi * 4)
    if i == 2:
        if x > 0.25:
            zx = -np.sin(x * np.pi * 4)
    if i == 3:
        if x <= 0.5 / 3.:
            zx = np.sin(x * np.pi * 6)
    if i == 4:
        if x > 0.5 / 3. and x <= 1 / 3.:
            zx = -np.sin(x * np.pi * 6)
    if i ==5:
        if x > 1/3.:
            zx = np.sin(x * np.pi * 6)

    if j == 0:
        zy = np.sin(y * np.pi * 2)

    if j == 1:
        if y <= 0.25:
            zy = np.sin(y * np.pi * 4)

    if j == 2:
        if y > 0.25:
            zy = -np.sin(y * np.pi * 4)
    
    if j == 3:
        if y <= 0.5 / 3.:
            zy = np.sin(y * np.pi * 6)

    if j == 4:
        if y > 0.5 / 3. and y <= 1 / 3.:
            zy = -np.sin(y * np.pi * 6)
    if j == 5:
        if y > 1/3.:
            zy = np.sin(y * np.pi * 6)

    return zx * zy


# Def random:

def Phiz(point, n, domain_index, result):
    x, y, z = point

    res = 0j
    if z == 0.5 and (x <= 0.5) and (y <= 0.5):
        for ii in range(6):
            for jj in range(6):
                res += Random[ii,jj] * function(x, y, ii,jj)	
    result[0] = res



def perturbate(grid, t, kappa_pert=None):
    P1 = bempp.api.function_space(grid, 'B-P', 1)
    grid_funz = bempp.api.GridFunction(P1, fun = Phiz)
    elements = grid.leaf_view.elements
    vertices = grid.leaf_view.vertices
    normals = P1.global_dof_normals
    x, y, z = vertices
    
    vertices[2, :] = z + t * grid_funz.coefficients
    return bempp.api.grid_from_element_data(vertices, elements)



def get_base_grid(h):

    gr0 = bempp.api.shapes.reentrant_cube(h=h, refinement_factor=1) 
    elements0 = list(gr0.leaf_view.entity_iterator(0))
    N0 = len(elements0)
    tol = h / 10.
    for i in range(N0):
        el0 = elements0[i]
        z = el0.geometry.corners[2]
        if np.linalg.norm(np.array([1/2, 1/2, 1/2]) - z) < tol:
            gr0.mark(el0)
    gr1 = gr0.refine()


    elements0 = list(gr1.leaf_view.entity_iterator(0))
    N0 = len(elements0)
    for i in range(N0):
        el0 = elements0[i]
        z = el0.geometry.corners[2]
        if np.linalg.norm(np.array([1/2, 1/2, 1/2])-z) < tol:
            gr1.mark(el0)

    base_grid = gr1.refine()
    return base_grid



config = bemppUQ.config.set_case('A')
precision = 10

h = 2.0 * np.pi / (precision * config["k_int"])



base_grid = get_base_grid(h=h) 


In [3]:
N = 2 * base_grid.leaf_view.entity_count(1)
print(N)
transmission_operators = assemble_operators(base_grid, config)
Umean, solution = evaluate_far_field(transmission_operators, config)


3204
iteration - 1 || residual - 0.145570369404 2.203482151031494
iteration - 2 || residual - 0.0303848264104 0.8174431324005127
iteration - 3 || residual - 0.00667101666613 0.7677147388458252
iteration - 4 || residual - 0.00108642514403 0.7753250598907471
iteration - 5 || residual - 0.000218854097476 0.7949249744415283
iteration - 6 || residual - 4.56938052444e-05 0.8346092700958252
iteration - 7 || residual - 9.49719737069e-06 0.2756805419921875


In [4]:

#far_field_p, sol_p = evaluate_far_field_sd(base_grid, transmission_operators, config, solution, grid_fun)

nrank = 6
lhs_op_list = []
rhs_list = []
for ii in range(nrank):
    for jj in range(nrank):
        print(ii,jj)
        def fun(point, n, domain_index, result):
            x, y, zip = point

            res = 0j
            if z == 0.5 and (x <= 0.5) and (y <= 0.5):
                res += function(x, y, ii,jj)
            result[0] = res
            
        space = bempp.api.function_space(base_grid, 'B-P', 1)
        grid_fun = bempp.api.GridFunction(space, fun= fun)
        
        lhs_op, rhs = evaluate_far_field_sd(base_grid, transmission_operators, config, solution, grid_fun)
        lhs_op_list.append(lhs_op)
        rhs_list.append(rhs)
            
            


0 0
iteration - 1 || residual - 0.0730059032248 3.6528687477111816
iteration - 2 || residual - 0.0101827662504 1.4814002513885498
iteration - 3 || residual - 0.0017069892295 1.5770857334136963
iteration - 4 || residual - 0.000300481952163 2.6599478721618652
iteration - 5 || residual - 6.5042355071e-05 2.014631748199463
iteration - 6 || residual - 1.10586679083e-05 2.006770372390747
iteration - 7 || residual - 2.01329955872e-06 0.14216923713684082
0 1
iteration - 1 || residual - 0.0661411133834 2.4024713039398193
iteration - 2 || residual - 0.00695175782676 0.8442428112030029
iteration - 3 || residual - 0.00114380845782 1.1140415668487549
iteration - 4 || residual - 0.0002039773576 1.078552484512329
iteration - 5 || residual - 4.20749009693e-05 1.2590034008026123
iteration - 6 || residual - 7.81858239655e-06 0.19379281997680664
0 2
iteration - 1 || residual - 0.0620910803891 1.2536139488220215
iteration - 2 || residual - 0.00697299130463 0.6452395915985107
iteration - 3 || residual - 0.

3 3
iteration - 1 || residual - 0.0780886842753 1.3349308967590332
iteration - 2 || residual - 0.00684934320509 0.706066370010376
iteration - 3 || residual - 0.00086409449102 0.6964530944824219
iteration - 4 || residual - 0.000129973483865 0.6725027561187744
iteration - 5 || residual - 2.07825669626e-05 0.70932936668396
iteration - 6 || residual - 3.95183823556e-06 0.2119159698486328
3 4
iteration - 1 || residual - 0.0815110226737 1.3362295627593994
iteration - 2 || residual - 0.00795007862351 0.6789658069610596
iteration - 3 || residual - 0.000787726124452 0.696021318435669
iteration - 4 || residual - 0.000109655481512 0.5834839344024658
iteration - 5 || residual - 1.85846992805e-05 0.5644290447235107
iteration - 6 || residual - 3.57811529606e-06 0.07307100296020508
3 5
iteration - 1 || residual - 0.08617941706 1.883549690246582
iteration - 2 || residual - 0.00831934556534 0.8546974658966064
iteration - 3 || residual - 0.000895656235084 1.0449700355529785
iteration - 4 || residual - 0

In [5]:
n = config['number_of_angles']

Sigma = np.zeros((n, n), dtype=np.complex128)
for ii in range(6):
    for jj in range(6):
        ff = np.sum(far_field_p, axis=0)
        Sigma += bemppUQ.utils.login.tensorize(ff,ff)
        

NameError: name 'far_field_p' is not defined

In [ ]:
t = .05
Uvar = t ** 2 * Sigma.diagonal()

In [ ]:
Umean = np.sum(Umean, axis=0)

In [ ]:
from matplotlib import pyplot as plt
from bemppUQ.utils.login import tensorize

fig, ax  = plt.subplots(1,1, figsize=(6,6))

FFp = Umean  + np.sqrt(np.abs(Uvar))
FFm = Umean  - np.sqrt(np.abs(Uvar))

angles = config['angles']

cs = 10 * np.log10(4 * np.pi * np.abs(Umean)**2)
cs_FOAp = 10 * np.log10(4 * np.pi * np.abs(FFp)**2)
cs_FOAm = 10 * np.log10(4 * np.pi * np.abs(FFm)**2)


ax.plot(angles, cs, '-r', label='Mean')
ax.plot(angles,cs_FOAp, '--b', label='FOAp')
ax.plot(angles,cs_FOAm, '--b', label='FOAm')

ax.set_title(r'$\mathrm{RCS}_z(\theta)$: BEM (red) \pm analytic (blue)', fontsize=14)

ax.set_xlabel(r'$\theta$ (rad)', fontsize=14)
ax.set_ylabel(r'$\mathrm{RCS}_z(\theta)$', fontsize=14)

ax.legend()
ax.grid()
